In [5]:
import pandas as pd
data=pd.read_csv('lakdiPrediction.csv')
data

,PRODUCT ID,CHAIR TYPE,WOOD,LENGTH(INCHES),WIDTH(INCHES),HEIGHT(INCHES),WOOD PRICE(PER CUBIC),QUANTITY OF WOOD USED(IN CUBIC),LABOUR CHARGE,PRICE
0,IRI277,DINING CHAIR,INDIAN ROSEWOOD,34,18,16,2400,2.0,773,5573
1,ELQ539,BAR CHAIR,TEAK,18,18,28,1400,1.5,516,2616
2,MEI548,DINING CHAIR,TEAK,34,18,16,1400,2.0,568,3368
3,HFH347,ARM CHAIR,INDIAN ROSEWOOD,35,35,36,2400,3.0,2033,9233
4,RNH631,BAR CHAIR,ACACIA,18,18,28,800,1.5,568,1768
...,...,...,...,...,...,...,...,...,...,...
1995,FQB404,DINING CHAIR,ACACIA,34,18,16,800,2.0,647,2247
1996,HFR309,RECLINER CHAIR,INDIAN ROSEWOOD,48,26,28,2400,3.0,1983,9183
1997,RMV760,ARM CHAIR,INDIAN ROSEWOOD,35,35,36,2400,3.0,2187,9387
1998,IPP155,BAR CHAIR,INDIAN ROSEWOOD,18,18,28,2400,1.5,535,4135


In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

data=synthetic_data

# Encode categorical variables
label_encoder_chair = LabelEncoder()
label_encoder_wood = LabelEncoder()

data['CHAIR TYPE'] = label_encoder_chair.fit_transform(data['CHAIR TYPE'])
data['WOOD'] = label_encoder_wood.fit_transform(data['WOOD'])

# Separate features and target
X = data.drop(columns=['PRICE', 'PRODUCT ID'])  # Features (exclude the target and irrelevant columns)
y = data['PRICE']  # Target (Price)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
# Train a Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Test the model and calculate accuracy (R^2 score)
r2_score = rf_model.score(X_test_scaled, y_test)
print(f'R^2 Score: {r2_score}')


R^2 Score: 0.9999960763579407


In [8]:
def validate_inputs(chair_type, wood_type, length, width, height, wood_quantity, labour_charge):
    # Ensure the chair type is valid
    if chair_type not in label_encoder_chair.classes_:
        raise ValueError(f"Invalid Chair Type! Available options are: {', '.join(label_encoder_chair.classes_)}")

    # Ensure the wood type is valid
    if wood_type not in label_encoder_wood.classes_:
        raise ValueError(f"Invalid Wood Type! Available options are: {', '.join(label_encoder_wood.classes_)}")

    # Ensure numerical values are positive and non-zero
    if length <= 0:
        raise ValueError("Length must be greater than zero.")
    if width <= 0:
        raise ValueError("Width must be greater than zero.")
    if height <= 0:
        raise ValueError("Height must be greater than zero.")
    if wood_quantity <= 0:
        raise ValueError("Wood quantity must be greater than zero.")
    if labour_charge <= 0:
        raise ValueError("Labour charge must be greater than zero.")

def predict_price():
    try:
        # Ensure that class labels are strings
        chair_classes = [str(cls) for cls in label_encoder_chair.classes_]
        wood_classes = [str(cls) for cls in label_encoder_wood.classes_]

        chair_type_input = input(f"Enter Chair Type ({', '.join(chair_classes)}): ").strip().upper()
        wood_type_input = input(f"Enter Wood Type ({', '.join(wood_classes)}): ").strip().upper()

        # Check if input is valid and handle exceptions for validation
        try:
            length = float(input("Enter Length (in inches): "))
            width = float(input("Enter Width (in inches): "))
            height = float(input("Enter Height (in inches): "))
            wood_quantity = float(input("Enter Quantity of Wood Used (in cubic feet): "))
            labour_charge = float(input("Enter Labour Charge: "))

            # Validate all inputs
            validate_inputs(chair_type_input, wood_type_input, length, width, height, wood_quantity, labour_charge)
        
        except ValueError as e:
            print(f"Input Error: {e}")
            return
        
        # Get wood price based on input wood type
        wood_prices = {'TEAK': 1400, 'ACACIA': 800, 'MAHAGONY': 800, 'INDIAN ROSEWOOD': 2400}
        wood_price = wood_prices.get(wood_type_input, 0)

        # Debug: Print the input values
        print(f'Debug Info - Chair Type: {chair_type_input}, Wood Type: {wood_type_input}, Length: {length}, Width: {width}, Height: {height}, Wood Price: {wood_price}, Wood Quantity: {wood_quantity}, Labour Charge: {labour_charge}')

        # Create new input data
        new_data = np.array([[label_encoder_chair.transform([chair_type_input])[0], 
                              label_encoder_wood.transform([wood_type_input])[0], 
                              length, width, height, wood_price, wood_quantity, labour_charge]])
        
        # Debug: Print the new data before scaling
        print(f'Debug Info - New Data (before scaling): {new_data}')

        # Standardize the new data
        new_data_scaled = scaler.transform(new_data)
        
        # Debug: Print the new data after scaling
        print(f'Debug Info - New Data (after scaling): {new_data_scaled}')
        
        # Predict the price
        predicted_price = rf_model.predict(new_data_scaled)
        print(f'Predicted Price: ₹{predicted_price[0]}')
        
    except ValueError as e:
        print(f"Error in processing input: {e}")
    except TypeError as e:
        print(f"Type Error: {e}")

# Call the function to predict the price based on user input
predict_price()


Enter Chair Type (ARM CHAIR, BAR CHAIR, DINING CHAIR, LOUNGE CHAIR, RECLINER CHAIR, ROCKING CHAIR):  DINING CHAIR
Enter Wood Type (ACACIA, INDIAN ROSEWOOD, MAHAGONY, TEAK):  INDIAN ROSEWOOD
Enter Length (in inches):  34
Enter Width (in inches):  18
Enter Height (in inches):  16
Enter Quantity of Wood Used (in cubic feet):  2
Enter Labour Charge:  773


Debug Info - Chair Type: DINING CHAIR, Wood Type: INDIAN ROSEWOOD, Length: 34.0, Width: 18.0, Height: 16.0, Wood Price: 2400, Wood Quantity: 2.0, Labour Charge: 773.0
Debug Info - New Data (before scaling): [[2.00e+00 1.00e+00 3.40e+01 1.80e+01 1.60e+01 2.40e+03 2.00e+00 7.73e+02]]
Debug Info - New Data (after scaling): [[-0.2783673  -0.4775542   0.10102943 -1.1992487  -1.73314883  1.68700281
  -0.90100672 -0.90732253]]
Predicted Price: ₹5572.76


C:\Users\Asus\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
